Run this script and while you are running this streaming diarizaiton module, run view_streaming_transcript.ipynb to view the transcript in real time.

In [ ]:
from streaming_asr_and_diar import ASR_DIAR_ONLINE, OnlineClusteringDiarizer, callback_sim, get_session_list
import argparse
from scipy.io import wavfile
import hydra
import numpy as np
import sys,os,argparse
from IPython.display import HTML
CONFIG_FILE = '.config_ipynb'
if os.path.isfile(CONFIG_FILE):
    with open(CONFIG_FILE) as f:
        sys.argv = f.read().split()
else:
    sys.argv = ['streaming_diarization_tutorial.py', '--device', '0']

# You must change these varialbes
SPK_EMBED_MODEL="/disk2/ejrvs/model_comparision/titanet-l.nemo"
OUTPUT_DIR="/home/taejinp/projects/run_time/streaming_diar_output_100_200"
# single_audio_file_path='/disk2/datasets/modified_callhome/callhome_16k/en_4092.wav'
single_audio_file_path='/home/taejinp/Downloads/arm_talk_16k.wav'

parser = argparse.ArgumentParser()
parser.add_argument("--pretrained_speaker_model", default=SPK_EMBED_MODEL, type=str, help="")
parser.add_argument("--device", default=0, type=int, help="")
parser.add_argument("--single_audio_file_path", default=None, type=str, help="")
parser.add_argument("--single_rttm_file_path", default=None, type=str, help="")
parser.add_argument("--audiofile_list_path", default=None, type=str, help="")
parser.add_argument("--reference_rttmfile_list_path", default=None, type=str, help="")
parser.add_argument("--diarizer_out_dir", default=OUTPUT_DIR, type=str, help="")
parser.add_argument("--color", default=True, type=str, help="")
parser.add_argument("--print_time", default=True, type=str, help="")
parser.add_argument("--force_real_time", default=True, type=str, help="")
parser.add_argument("--frame_len", default=1, type=int, help="")
parser.add_argument("--frame_overlap", default=2, type=int, help="")
parser.add_argument("--round_float", default=2, type=int, help="")
parser.add_argument("--window_length_in_sec", default=1.5, type=float, help="")
parser.add_argument("--shift_length_in_sec", default=0.75, type=float, help="")
parser.add_argument("--print_transcript", default=False, type=bool, help="")
parser.add_argument("--use_lm_for_asr", default=False, type=bool, help="")
parser.add_argument("--lenient_overlap_WDER", default=True, type=bool, help="")
parser.add_argument("--speech_detection_threshold", default=100, type=int, help="")
parser.add_argument("--ASR_model_name", default='stt_en_conformer_ctc_large', type=str, help="")
parser.add_argument("--sample_rate", default=16000, type=int, help="")
parser.add_argument("--online_diar_buffer_segment_quantity", default=200, type=int, help="")
parser.add_argument("--online_history_buffer_segment_quantity", default=100, type=int, help="")
parser.add_argument("--enhanced_count_thres", default=80, type=int, help="")
parser.add_argument("--max_num_speaker", default=8, type=int, help="")
args = parser.parse_args()

args.single_audio_file_path = single_audio_file_path
overrides = [
f"diarizer.speaker_embeddings.model_path={args.pretrained_speaker_model}",
f"diarizer.path2groundtruth_rttm_files={args.reference_rttmfile_list_path}",
f"diarizer.paths2audio_files={args.audiofile_list_path}",
f"diarizer.out_dir={args.diarizer_out_dir}",
f"diarizer.speaker_embeddings.window_length_in_sec={args.window_length_in_sec}",
f"diarizer.speaker_embeddings.shift_length_in_sec={args.shift_length_in_sec}",
]

params = vars(args)
hydra.initialize(config_path="conf")
cfg_diar = hydra.compose(config_name="/speaker_diarization.yaml", overrides=overrides)
diar_init = OnlineClusteringDiarizer(cfg=cfg_diar, params=params)
diar_init.prepare_diarization()
diar_init, session_list = get_session_list(diar_init, args)
for uniq_key in session_list:
    diar = OnlineClusteringDiarizer(cfg=cfg_diar, params=params)
    diar.uniq_key = uniq_key 
    diar.prepare_diarization()
    asr = ASR_DIAR_ONLINE(diar, params, offset=4)
    asr.reset()

    dcont = diar_init.AUDIO_RTTM_MAP[uniq_key]
    samplerate, sdata = wavfile.read(dcont['audio_path'])
    asr.rttm_file_path = dcont['rttm_path']

    filePath = f"{asr.diar._out_dir}/{uniq_key}.csv"
    if os.path.exists(filePath): 
        os.remove(filePath)
    # input("Press Enter to continue...")
    for i in range(int(np.floor(sdata.shape[0]/asr.n_frame_len))):
        callback_sim(asr, uniq_key, i, sdata, frame_count=None, time_info=None, status=None)

[NeMo W 2021-10-18 16:58:04 optimizers:47] Apex was not found. Using the lamb optimizer will error out.
[NeMo W 2021-10-18 16:58:06 experimental:27] Module <class 'nemo.collections.asr.data.audio_to_text_dali._AudioTextDALIDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
Global seed set to 42
[NeMo W 2021-10-18 16:58:07 nemo_logging:349] /home/taejinp/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:83: UserWarning: 
        Found GPU%d %s which is of cuda capability %d.%d.
        PyTorch no longer supports this GPU because it is too old.
        The minimum cuda capability supported by this library is %d.%d.
        
      warnings.warn(old_gpu_warn.format(d, name, major, minor, min_arch // 10, min_arch % 10))
    
[NeMo W 2021-10-18 16:58:07 modelPT:130] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
 

[NeMo I 2021-10-18 16:58:07 features:262] PADDING: 16
[NeMo I 2021-10-18 16:58:07 features:279] STFT using torch
[NeMo I 2021-10-18 16:58:07 label_models:90] Training with Angular Softmax Loss
[NeMo I 2021-10-18 16:58:10 save_restore_connector:143] Model ExtractSpeakerEmbeddingsModel was successfully restored from /disk2/ejrvs/model_comparision/titanet-l.nemo.
[NeMo I 2021-10-18 16:58:10 clustering_diarizer:120] Speaker Model restored locally from /disk2/ejrvs/model_comparision/titanet-l.nemo
[NeMo I 2021-10-18 16:58:10 streaming_asr_and_diar:516] Setting online diarization buffer to : 100
[NeMo I 2021-10-18 16:58:10 streaming_asr_and_diar:526] Setting online diarization buffer to : 200


[NeMo W 2021-10-18 16:58:11 modelPT:130] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 3
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2021-10-18 16:58:11 modelPT:137] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_valida

[NeMo I 2021-10-18 16:58:11 features:262] PADDING: 16
[NeMo I 2021-10-18 16:58:11 features:279] STFT using torch
[NeMo I 2021-10-18 16:58:11 label_models:90] Training with Angular Softmax Loss
[NeMo I 2021-10-18 16:58:11 save_restore_connector:143] Model ExtractSpeakerEmbeddingsModel was successfully restored from /disk2/ejrvs/model_comparision/titanet-l.nemo.
[NeMo I 2021-10-18 16:58:11 clustering_diarizer:120] Speaker Model restored locally from /disk2/ejrvs/model_comparision/titanet-l.nemo
[NeMo I 2021-10-18 16:58:11 streaming_asr_and_diar:516] Setting online diarization buffer to : 100
[NeMo I 2021-10-18 16:58:11 streaming_asr_and_diar:526] Setting online diarization buffer to : 200
[NeMo I 2021-10-18 16:58:11 cloud:56] Found existing object /home/taejinp/.cache/torch/NeMo/NeMo_1.3.0/stt_en_conformer_ctc_large/b2da9ca277cfb30bb0c6b2abd44eef02/stt_en_conformer_ctc_large.nemo.
[NeMo I 2021-10-18 16:58:11 cloud:62] Re-using file from: /home/taejinp/.cache/torch/NeMo/NeMo_1.3.0/stt_en_

[NeMo W 2021-10-18 16:58:14 modelPT:130] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data/nemo_asr_set/asr_set_1.4/train_no_appen/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 8
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    shuffle_n: 2048
    is_tarred: true
    tarred_audio_filepaths: /data/nemo_asr_set/asr_set_1.4/train_no_appen/audio__OP_0..2047_CL_.tar
    
[NeMo W 2021-10-18 16:58:14 modelPT:137] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath:
    - /manifests/librispeech/l

...

216.97ms 'run_ASR_model'
0.05ms '_get_speech_labels_from_decoded_prediction'
0.56ms '_get_diar_segments'
2.42ms 'getReducedMat'
1.33ms 'matchNewOldclusterLabels'
97.29ms '_online_diarization'
[NeMo I 2021-10-18 17:12:34 streaming_asr_and_diar:1361] Streaming Diar [arm_talk_16k][frame-  859th  ]:


[NeMo W 2021-10-18 17:12:35 nmse_clustering:276] Graph is not fully connected and the clustering result might not be accurate.


205.82ms 'run_ASR_model'
0.07ms '_get_speech_labels_from_decoded_prediction'
0.70ms '_get_diar_segments'
3.53ms 'getReducedMat'
1.30ms 'matchNewOldclusterLabels'
82.87ms '_online_diarization'
[NeMo I 2021-10-18 17:12:35 streaming_asr_and_diar:1361] Streaming Diar [arm_talk_16k][frame-  860th  ]:


[NeMo W 2021-10-18 17:12:36 nmse_clustering:276] Graph is not fully connected and the clustering result might not be accurate.


228.85ms 'run_ASR_model'
0.05ms '_get_speech_labels_from_decoded_prediction'
0.73ms '_get_diar_segments'
3.56ms 'getReducedMat'
1.34ms 'matchNewOldclusterLabels'
94.68ms '_online_diarization'
[NeMo I 2021-10-18 17:12:36 streaming_asr_and_diar:1361] Streaming Diar [arm_talk_16k][frame-  861th  ]:
